In [20]:
import os 
import sys

In [21]:
os.getcwd()

'/home/owner/Documents/Github_projects/Tinder_dash_demo/Notebooks'

In [22]:
sys.path.append('/home/owner/Documents/Github_projects/Tinder_dash_demo/Scripts/')
sys.path.append('/home/owner/Documents/Github_projects/Tinder_dash_demo/')

In [61]:
import dash_table

In [107]:
import dash
import pandas as pd
import numpy as np
import json
from Scripts import message_df_fx as msg_fx


In [24]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


In [28]:
data_path = '/home/owner/Documents/Github_projects/Tinder_dash_demo/Data/data.json'

In [29]:
with open(data_path, "rb") as inp: 
    data = json.load(inp)

In [30]:
list_of_dfs = [msg_fx.get_msg_df(msg_dict) for msg_dict in data["Messages"]]
all_msg_df = pd.concat(list_of_dfs, axis=0)


In [31]:
all_msg_df['date'] = all_msg_df['sent_date'].dt.date

In [32]:
all_msg_df['date2'] = all_msg_df['sent_date'].apply(msg_fx.flatten_date)

In [35]:
flag_col = ['explicit_word_in_msg', 'funny_word_in_msg', 'question_mark_in_msg', 'question_word_in_msg',
            "exclamation_mark_in_msg"]

In [45]:
    dt_gb = all_msg_df.groupby('date')
    n_msg_over_time = dt_gb.apply(len)

In [46]:
n_msg_over_time.head()

date
2014-03-26    4
2014-03-27    3
2014-04-02    1
2014-04-07    5
2014-06-07    1
dtype: int64

In [51]:
def create_word_per_message_graph():
    dt_gb = all_msg_df.groupby('date')
    n_msg_over_time = dt_gb.apply(len)
    total_trace = go.Scatter(
        x=n_msg_over_time.index, 
        y=n_msg_over_time.values,
        name="Total Number of Messages"
        )
    print(n_msg_over_time.shape)

    def create_plots(flag_over_time, flag_name):
        trace = go.Scatter(
            x=flag_over_time.index,
            y=flag_over_time,
            name= flag_name
        )
        return(trace)
    traces = [create_plots(dt_gb[flag].sum(), flag) for flag in flag_col]
    traces.insert(0,total_trace)

    layout = dict(title = 'Number of Message Types over Time',
                  xaxis = dict(title='Date'),
                  yaxis = dict(title='Number of Messages')
                  )
    fig = go.Figure(data= traces, layout=layout)
    return(fig)


In [52]:
fig1 = create_word_per_message_graph()

(214,)


In [53]:
iplot(fig1)

## Dash example of a table 

In [62]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')

app = dash.Dash(__name__)

app.layout = dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in df.columns],
    data=df.to_dict("rows"),
)


In [64]:
df.head()

,State,Number of Solar Plants,Installed Capacity (MW),Average MW Per Plant,Generation (GWh)
0,California,289,4395,15.3,10826
1,Arizona,48,1078,22.5,2550
2,Nevada,11,238,21.6,557
3,New Mexico,33,261,7.9,590
4,Colorado,20,118,5.9,235


## Apply table knowledge

In [55]:
from Scripts import usage_analysis_fx as usage


In [ ]:
usage_df = pd.DataFrame(data["Usage"])


In [58]:
max_usage = usage.gather_max_usage(usage_df)

In [76]:
max_usage

,metric,date,max of index
0,app_opens,2017-03-06,348
1,matches,2015-09-25,10
2,messages_received,2015-09-25,26
3,messages_sent,2015-09-25,36
4,swipes_likes,2015-09-26,153
5,swipes_passes,2016-06-12,87


In [81]:
trace_tbl = go.Table(header=dict(values=max_usage.columns), 
                     cells=dict(values=max_usage.values.T),
                     name="Max Usage Metrics")
data = [trace_tbl]

In [82]:
iplot(data)

In [93]:
derived_metrics = usage.gather_usage_stats(usage_df)


In [94]:
derived_metrics = pd.Series(derived_metrics)

In [95]:
derived_metrics

like_to_pass_ratio           3.73
n_avg_msg_rec_per_match      3.13
n_avg_msg_sent_per_match     3.87
swipes/app_open              4.21
swipes_per_act_day          24.28
swipes_per_tot_cal_day       11.0
total_swipes                16196
dtype: object

In [103]:
trace_tbl = go.Table(header=dict(values=derived_metrics.index), 
                     cells=dict(values=derived_metrics.values),
                     name="Derived Usage Metrics")
data = [trace_tbl]
layout = dict(width=1300, height=300)
fig = dict(data=data, layout=layout)

In [104]:
iplot(fig)

In [108]:
usage_df2 = usage_df.copy()
usage_df2.index = pd.to_datetime(usage_df.index, format="%Y-%m-%d")
usage_df2.index = pd.Series(usage_df2.index).apply(msg_fx.flatten_date)

# Create aggregated time series data and plots
funcs = {"mean": np.mean, "max": np.max}  # , "std":np.std}
for usage_col in usage_df2.columns:
    gb_dt = pd.Series(usage_df2[usage_col]).groupby(usage_df2.index)
    agg_series = {usage_col + " " + func_names: gb_dt.apply(func) for func_names, func in funcs.items()}

In [125]:
traces = []
for usage_col in usage_df2.columns:
    gb_dt = pd.Series(usage_df2[usage_col]).groupby(usage_df2.index)
    temp_traces = []
    for func_names, func in funcs.items():
        temp_series = gb_dt.apply(func)
        temp_name = usage_col + " " + func_names
        temp_traces.append(create_plots(temp_series, temp_name))
    traces.append(temp_traces)

In [126]:
len(traces)

6

In [127]:
for trace2 in traces:
    iplot(trace2)

In [124]:
agg_series

{'swipes_passes mean': 201411     0.555556
 201412     0.625000
 201501     0.190476
 201502     0.100000
 201503     0.625000
 201504     1.333333
 201505     0.333333
 201506     0.333333
 201507     1.875000
 201508     2.375000
 201509     0.850000
 201510     1.125000
 201511     0.272727
 201512     2.923077
 201601     2.888889
 201602     2.434783
 201603     7.791667
 201604    14.230769
 201605    13.500000
 201606    14.350000
 201607     4.125000
 201608     0.000000
 201611    15.000000
 201612     6.608696
 201701     9.869565
 201702     3.400000
 201703     6.428571
 201704     3.363636
 201705     4.750000
 201706     1.384615
 201707     3.800000
 201708     4.437500
 201709     1.666667
 201710     2.142857
 201801    34.750000
 201802     5.520000
 201803     4.750000
 201804     4.869565
 201805     5.038462
 201806     4.476190
 201807     4.066667
 201808     1.000000
 201809     3.583333
 201810     9.653846
 201811    10.800000
 201812    14.000000
 Name: swipe

In [120]:
def create_plots(flag_over_time, flag_name):
    trace = go.Scatter(
            x=flag_over_time.index,
            y=flag_over_time,
            name= flag_name
        )
    return(trace)
